In [265]:
import random
import gurobipy as gp
from gurobipy import GRB
import copy
import numpy as np

n=10
def generate_data(n):
    # Initializing an empty adjacency matrix with zeros
    W = [[0] * n for _ in range(n)]

    # Generate random edge weights (assuming non-negative weights)
    for i in range(n):
        for j in range(i+1, n):
            t = random.randint(0,2)
            if t == 0:
                weight = 0
            else:
                weight = random.randint(1,10)
            # weight = random.uniform(0, 2)  # Adjust the range as needed
            # Ensure that edges are undirected by setting W[i][j] = W[j][i] = weight
            W[i][j] = W[j][i] = weight
    return W

In [266]:
def BP(W,n):
    # Create a new model
    m = gp.Model("mip1")
    # Create variable

    x = {}
    x = m.addVars(range(n),vtype = "B", name="x")
    y = m.addVars(range(n),range(n),vtype = "B", name="y")

    m.setObjective(sum(W[i][j]*y[i,j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)
    m.addConstrs(y[i,j]<=(x[i]+x[j]) for i in range(n) for j in range(n))
    m.addConstrs(y[i,j]+x[i]+x[j]<=2 for i in range(n) for j in range(n))
    m.optimize()
    y_result=[]

    for v in x.values():
        if v.X == 0 or v.X==-0:
            y_result.append(0)
        else:
            y_result.append(1)
    return y_result

In [267]:
def count_weight(W:list, A:list, B:list):
    total_weight = 0
    for i in A:
        for j in B:
            total_weight += W[i][j]
    return total_weight

In [268]:
def weight_change(W, A, B, current_weight):
    weight_change_list = []
    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        if i in A:

            temp_B.append(i)
            temp_A.remove(i)
        else:

            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        result = new_weight - current_weight
        weight_change_list.append(result)
    return weight_change_list

In [269]:
def random_set():
    A = []
    B = []
    xdata_gh = []

    for i in range(n):
        r = random.randint(0,1)
        if r == 0:
            xdata_gh.append(0)
            A.append(i)
        else:
            xdata_gh.append(1)
            B.append(i)
    return xdata_gh, A, B

In [270]:
def GH(W, A, B, n):
    total_weight = count_weight(W, A, B)

    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        if i in A:
            temp_B.append(i)
            temp_A.remove(i)
        else:
            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        if new_weight >= total_weight:
            A = temp_A
            B = temp_B
            total_weight = new_weight
    A = sorted(A)
    B = sorted(B)
    total_weight = count_weight(W, A, B)
    return total_weight, A, B

In [271]:
# k=5

# A_GH_list = []
# B_GH_list = []
# gh_weight_list = []
# raw_data = []
# xdata1_l = []
# ydata1_l = []

# for i in range(k):
#     sum_W = []
#     data_list = []
#     data = generate_data(n)

#     xdata_gh, A, B = random_set()
#     gh_weight, A_GH, B_GH = GH(data,A,B,n)

#     A_GH_list.append(A_GH)
#     B_GH_list.append(B_GH)
#     gh_weight_list.append(gh_weight)

#     current_weight = count_weight(data,A,B)
#     weight_diff = weight_change(data,A,B,current_weight)

#     # result = BP(data, n)
    # ydata1_l.append(result)
    
    # for j in data:
    #     sum_W.append(sum(j))

    # for j in range(len(sum_W)):
    #     test = []
    #     test.append(sum_W[j])
    #     test.append(xdata_gh[j])
    #     test.append(weight_diff[j])
    #     xdata1_l.append(test)
    # raw_data.append(data)

In [272]:
def get_data(k):
    A_GH_list = []
    B_GH_list = []
    gh_weight_list = []
    raw_data = []
    xdata1_l = []
    ydata1_l = []

    for i in range(k):
        sum_W = []
        
        data = generate_data(n)

        connected_nodes = []
        for v in range(len(data)):
            connect = 0
            for z in data[v]:
                if z>0:
                    connect+=1
            connected_nodes.append(connect)

        xdata_gh, A, B = random_set()
        gh_weight, A_GH, B_GH = GH(data,A,B,n)

        A_GH_list.append(A_GH)
        B_GH_list.append(B_GH)
        gh_weight_list.append(gh_weight)

        current_weight = count_weight(data,A,B)
        weight_diff = weight_change(data,A,B,current_weight)

        result = BP(data, n)
        for j in range(len(result)):
            data_list = []
            data_list.append(result[j])
            ydata1_l.append(data_list)
        
        for j in data:
            sum_W.append(sum(j))
        # data_list.append(sum_W)
        # data_list.append(xdata_gh)
        # data_list.append(weight_diff)
        # xdata1_l.append(data_list)

        for j in range(len(sum_W)):
            test = []
            # for z in data[j]:
            #     test.append(z)
            test.append(sum_W[j])
            test.append(xdata_gh[j])
            test.append(weight_diff[j])
            test.append(connected_nodes[j])
            xdata1_l.append(test)

    return xdata1_l, ydata1_l, A_GH_list, B_GH_list, gh_weight_list, raw_data

In [273]:
k1 = 10
xtrain_list, ytrain_list, A_GH_train, B_GH_train, gh_weight_train, raw_data = get_data(k1)

k2 = 5
xtest_list, ytest_list, A_GH_test, B_GH_test, gh_weight_test, raw_data = get_data(k2)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 200 rows, 110 columns and 580 nonzeros
Model fingerprint: 0x7c40925b
Variable types: 0 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 144 rows and 72 columns
Presolve time: 0.00s
Presolved: 56 rows, 38 columns, 168 nonzeros
Variable types: 0 continuous, 38 integer (38 binary)
Found heuristic solution: objective 186.0000000

Root relaxation: objective 3.340000e+02, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 200 rows, 110 columns and 580 nonzeros
Model fingerprint: 0x757be6f3
Variable types: 0 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 134 rows and 67 columns
Presolve time: 0.01s
Presolved: 66 rows, 43 columns, 198 nonzeros
Variable types: 0 continuous, 43 integer (43 binary)
Found heuristic solution: objective 150.0000000

Root relaxation: objective 2.740000e+02, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  274.00000    0   10  15

In [274]:
ytrain_list

[[0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [0],
 [1]]

In [275]:
ytest_list

[[1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1]]

In [276]:
xtrain = np.array(xtrain_list)
xtest = np.array(xtest_list)
ytrain = np.array(ytrain_list)
ytest = np.array(ytest_list)

In [277]:
def precision(y_true, y_pred):

    from tensorflow.keras import backend as K
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):

    from tensorflow.keras import backend as K
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta(y_true, y_pred, beta=1):

    from tensorflow.keras import backend as K
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    #if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
    #    return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def f1score(y_true, y_pred):

    return fbeta(y_true, y_pred, beta=1)

In [278]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(100, 4)
(50, 4)
(100, 1)
(50, 1)


In [281]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import datetime
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
tf.random.set_seed(42)
log_dir = "logs/fit/" + "single_layer"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

xtrain=np.reshape(xtrain,[k1, 4*n])
xtest=np.reshape(xtest,[k2, 4*n])

input_shape = (4*n, )
nrlayers = 4
nrnodes = [ 70, 60, 30, 20]
rate = 0.002

model = keras.Sequential()
numclasses = 1

for i in range(nrlayers + 1):
    if (i == 0):
        # input layer
        model.add(keras.Input(shape=input_shape))
    else:
        # hidden layers
        nrnode = nrnodes[i - 1]
        model.add(keras.layers.Dense(nrnode, activation="relu"))

model.add(keras.layers.Dense(numclasses, activation="sigmoid"))

model.summary()

batch_size = 200 # Based on SGD --> 1 gradient is calculated using 32 data points every time
#so in each epoch, training_size/batch_size many gradient calculations and weight updates
epochs = 50 # How many times to go through the data to complete the SGD

opt = keras.optimizers.SGD(learning_rate=rate)
model.compile(loss="binary_crossentropy",
                optimizer=opt,
                metrics=[f1score])

history = model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epochs, validation_split=0.1,)

score_train = model.evaluate(xtrain, ytrain, verbose=1)
print(f"Train loss: {score_train[0]}, Train accuracy:, {score_train[1]}")

score = model.evaluate(xtest, ytest, verbose=1)
print(f"Test loss: {score[0]}, Test accuracy:, {score[1]}")

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_61 (Dense)            (None, 70)                2870      
                                                                 
 dense_62 (Dense)            (None, 60)                4260      
                                                                 
 dense_63 (Dense)            (None, 30)                1830      
                                                                 
 dense_64 (Dense)            (None, 20)                620       
                                                                 
 dense_65 (Dense)            (None, 1)                 21        
                                                                 
Total params: 9601 (37.50 KB)
Trainable params: 9601 (37.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
1/1 [===

ValueError: Data cardinality is ambiguous:
  x sizes: 10
  y sizes: 100
Make sure all arrays contain the same number of samples.

In [ ]:
plt.plot(history.history['val_loss'], label='val_loss')
plt.plot(history.history['loss'], label='val')
plt.title('loss')
plt.legend()

In [ ]:
plt.plot(history.history['f1score'], label = 'f1')
plt.plot(history.history['val_f1score'], label = 'val_f1')
plt.title('model_f1')
plt.legend()

In [ ]:
pred_f = models1.predict(xtest)
pred = pred_f.tolist()
pred = [[round(x) for x in sublist] for sublist in pred]
print(accuracy_score(pred,ytest)*100)

2/2 [==============================] - 0s 4ms/step
44.0


In [ ]:
pred_f

array([[0.53943914],
       [0.54096043],
       [0.5402546 ],
       [0.5402475 ],
       [0.54027957],
       [0.5402549 ],
       [0.54026574],
       [0.54023   ],
       [0.5401212 ],
       [0.5329895 ],
       [0.53990793],
       [0.5402395 ],
       [0.5330722 ],
       [0.5379411 ],
       [0.53299415],
       [0.53717303],
       [0.5402281 ],
       [0.5348147 ],
       [0.5329949 ],
       [0.535841  ],
       [0.53312695],
       [0.54056144],
       [0.5440772 ],
       [0.53299814],
       [0.5329901 ],
       [0.53416985],
       [0.53327256],
       [0.5330583 ],
       [0.53407866],
       [0.5391637 ],
       [0.5404325 ],
       [0.53301203],
       [0.5402494 ],
       [0.53371924],
       [0.5332788 ],
       [0.535206  ],
       [0.54025066],
       [0.54025495],
       [0.5333689 ],
       [0.5395388 ],
       [0.5402507 ],
       [0.5402519 ],
       [0.5406921 ],
       [0.5335776 ],
       [0.53988034],
       [0.53854775],
       [0.540053  ],
       [0.534

In [ ]:
NN = models1.predict(xtest)
NN_ytest = NN.tolist()
NN_ytest = [[round(x) for x in sublist] for sublist in NN_ytest]

2/2 [==============================] - 0s 4ms/step


In [ ]:
A_NN = []
B_NN = []

for i in NN_ytest:
    temp_A = []
    temp_B = []
    for j in range(len(i)):
        if i[j]>0.5:
            temp_A.append(j)
        else:
            temp_B.append(j)
    A_NN.append(temp_A)
    B_NN.append(temp_B)

In [ ]:
NN_result = 0
GH_result = 0
for i in range(len(A_NN)):
    GH_weight = count_weight(raw_data[i], A_GH_test[i], B_GH_test[i])
    NN_weight = count_weight(raw_data[i], A_NN[i], B_NN[i])

    if NN_weight >= GH_weight:
        NN_result += 1
    else:
        GH_result += 1

print(f"Outcome of the MLGH where they have a higher performance is {NN_result/len(A_NN)}%")
print(f"Outcome of the GH where they have a higher performance is {GH_result/len(A_NN)}%")

IndexError: list index out of range